In [52]:
import numpy as np
from scipy.stats import entropy
from scipy.sparse import issparse
from joblib import Parallel, delayed
from scipy.sparse import csr_matrix

def mutual_information_matrix_serial(matrix, nbins=20, n_jobs=-1):
    """
    Computes the mutual information matrix in parallel, working directly with sparse matrices,
    and only computes the upper triangular part of the matrix.
    """
    if not issparse(matrix):
        matrix = csr_matrix(matrix)

    n_features = matrix.shape[0]
    mi_matrix = np.zeros((n_features, n_features))

    def compute_pairwise_mi(vi, vj, nbins=20):
        joint_counts, _, _ = np.histogram2d(vi, vj, bins=nbins)
        if joint_counts.sum() == 0:
            return 0  # No mutual information if no overlap
        joint_prob = joint_counts / (joint_counts.sum() + 1e-8)

        marginal_i = joint_prob.sum(axis=1) + 1e-8
        marginal_j = joint_prob.sum(axis=0) + 1e-8

        h_xy = entropy(joint_prob.flatten(), base=2)
        h_x = entropy(marginal_i, base=2)
        h_y = entropy(marginal_j, base=2)

        return float(h_x + h_y - h_xy)

    for i in range(n_features):
        for j in range(i, n_features):
            vi = matrix[i, :].toarray().flatten() if issparse(matrix) else matrix[i, :]
            vj = matrix[j, :].toarray().flatten() if issparse(matrix) else matrix[j, :]
            mi_matrix[i, j] = compute_pairwise_mi(vi, vj, nbins=nbins)
            if i != j:
                mi_matrix[j, i] = mi_matrix[i, j]  # Exploit symmetry
    return mi_matrix


In [53]:
from scipy.sparse import random as sparse_random
from scipy.io import mmwrite

# Generate a sparse random matrix with 1000 rows and 5000 columns
# Density of the matrix is set to 0.01 (1% non-zero elements)
sparse_matrix = sparse_random(5000, 5000, density=0.01, format='csr')


# vec1 = [1, 2, 3, 0, 0]  # Row 0
# vec2 = [4, 0, 6, 0, 0]  # Row 1
# vec3 = [0, 1, 3, 7, 9]  # Row 2
# vec4 = [5, 0, 0, 0 ,2] # Row 3
# matrix = [ vec1 , vec2, vec3, vec3]
# sparse_matrix = csr_matrix(matrix)
mmwrite("sparse_matrix.mtx", sparse_matrix)

In [44]:
mi_matrix = mutual_information_matrix_serial(sparse_matrix, nbins=20, n_jobs=-1)
print(mi_matrix)
sparse_matrix_mi = csr_matrix(mi_matrix)
print(sparse_matrix_mi)
mmwrite("sparse_matrix_mi.mtx", sparse_matrix_mi)

[[1.04511018e-01 1.39297936e-04 1.53754494e-04 ... 1.97565759e-04
  1.68225922e-04 1.24856218e-04]
 [1.39297936e-04 9.86257491e-02 1.38947486e-04 ... 1.78337823e-04
  9.53201558e-03 1.12988987e-04]
 [1.53754494e-04 1.38947486e-04 1.10020092e-01 ... 1.97215309e-04
  1.67875473e-04 1.88919906e-02]
 ...
 [1.97565759e-04 1.78337823e-04 1.97215309e-04 ... 1.38195074e-01
  2.16112186e-04 1.59479687e-04]
 [1.68225922e-04 9.53201558e-03 1.67875473e-04 ... 2.16112186e-04
  1.21412977e-01 1.36034421e-04]
 [1.24856218e-04 1.12988987e-04 1.88919906e-02 ... 1.59479687e-04
  1.36034421e-04 8.72299507e-02]]
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1000000 stored elements and shape (1000, 1000)>
  Coords	Values
  (0, 0)	0.10451101753386409
  (0, 1)	0.00013929793570788274
  (0, 2)	0.00015375449389490314
  (0, 3)	0.0001394731604646149
  (0, 4)	0.00013895608884953448
  (0, 5)	0.00016822592247336954
  (0, 6)	0.0002119006568598869
  (0, 7)	8.12779584359602e-05
  (0, 8)	0.00011008746315

In [54]:
from scipy.sparse import issparse
from scipy.stats import entropy
import numpy as np
from joblib import Parallel, delayed

def mutual_information_matrix_parallel(matrix, nbins=20, n_jobs=-1):
    """
    Computes the mutual information matrix in parallel, working directly with sparse matrices,
    and computes the full matrix (including the diagonal elements).
    """
    if not issparse(matrix):
        matrix = matrix.tocsr()

    n_features = matrix.shape[0]
    mi_matrix = np.zeros((n_features, n_features))

    def compute_pairwise_mi(i, j, matrix, nbins=20):
        """
        Computes mutual information between row i and row j of the sparse matrix.
        """
        vi = matrix[i, :].toarray().flatten() if issparse(matrix) else matrix[i, :]
        vj = matrix[j, :].toarray().flatten() if issparse(matrix) else matrix[j, :]
        
        joint_counts, _, _ = np.histogram2d(vi, vj, bins=nbins)
        if joint_counts.sum() == 0:
            return 0  # No mutual information if no overlap
        joint_prob = joint_counts / (joint_counts.sum() + 1e-8)

        marginal_i = joint_prob.sum(axis=1) + 1e-8
        marginal_j = joint_prob.sum(axis=0) + 1e-8

        h_xy = entropy(joint_prob.flatten(), base=2)
        h_x = entropy(marginal_i, base=2)
        h_y = entropy(marginal_j, base=2)

        return float(h_x + h_y - h_xy)

    # Parallelizing the pairwise mutual information computation
    jobs = [(i, j) for i in range(n_features) for j in range(i, n_features)]  # Includes diagonal
    results = Parallel(n_jobs=n_jobs)(
        delayed(compute_pairwise_mi)(i, j, matrix, nbins) for i, j in jobs
    )

    # Fill the matrix with the results
    for idx, (i, j) in enumerate(jobs):
        mi_matrix[i, j] = results[idx]
        mi_matrix[j, i] = results[idx]  # Exploit symmetry to avoid duplicate computation

    return mi_matrix


In [ ]:
#from scipy.sparse import random as sparse_random
mi_matrix = mutual_information_matrix_parallel(sparse_matrix, nbins=20, n_jobs=-1)
#print(mi_matrix)
sparse_matrix_mi = csr_matrix(mi_matrix)
print(sparse_matrix_mi)
mmwrite("sparse_matrix_mi.mtx", sparse_matrix_mi)

In [47]:
print(mi_matrix.shape)

(1000, 1000)
